<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Запросы из нескольких таблицами

## Легенда

### `clients` — список зарегистрированных клиентов:

| client\_id | name  |
| ---------- | ----- |
| 1          | Anna  |
| 2          | Boris |
| 3          | Clara |
| 4          | David |

---

### `orders` — список покупок:

| order\_id | client\_id | total\_uah |
| --------- | ---------- | ---------- |
| 101       | 1          | 500        |
| 102       | 3          | 800        |
| 103       | 3          | 1200       |
| 104       | 5          | 300        |

---


## JOIN

JOIN — это способ **“сшить”** строки из разных таблиц по общему признаку (чаще всего — идентификатору)

### Структура JOIN

```sql
SELECT ...
FROM таблица_1
[INNER] JOIN таблица_2
  ON таблица_1.ключ = таблица_2.ключ
```

* **JOIN** соединяет строки двух таблиц
* **ON** указывает, по какому признаку
* Выводятся **все строки**, где совпал ключ

## Типы JOIN и как они работают

| JOIN         | Что возвращает                                     |
| ------------ | -------------------------------------------------- |
| `INNER JOIN` | Только совпадающие строки из обеих таблиц          |
| `LEFT JOIN`  | Все строки из левой таблицы + совпадения из правой |
| `RIGHT JOIN` | Все строки из правой таблицы + совпадения из левой |
| `FULL JOIN`  | Все строки из обеих таблиц, даже без совпадений    |

---

### `INNER JOIN`

```sql
SELECT clients.client_id, clients.name, orders.total_uah
FROM clients
INNER JOIN orders
   ON clients.client_id = orders.client_id;
```

Покажет только тех клиентов, у кого есть хотя бы один заказ:

| client\_id | name  | total\_uah |
| ---------- | ----- | ---------- |
| 1          | Anna  | 500        |
| 3          | Clara | 800        |
| 3          | Clara | 1200       |

---

### `LEFT JOIN`

```sql
SELECT clients.client_id, clients.name, orders.total_uah
FROM clients
LEFT JOIN orders
   ON clients.client_id = orders.client_id;
```

Покажет всех клиентов, даже если у них **нет заказов** (в таком случае `total_uah` будет `NULL`):

| client\_id | name  | total\_uah |
| ---------- | ----- | ---------- |
| 1          | Anna  | 500        |
| 2          | Boris | NULL       |
| 3          | Clara | 800        |
| 3          | Clara | 1200       |
| 4          | David | NULL       |

---

### `RIGHT JOIN`

```sql
SELECT clients.client_id, clients.name, orders.total_uah
FROM clients
RIGHT JOIN orders
   ON clients.client_id = orders.client_id;
```

Покажет все заказы, даже если клиент **не зарегистрирован** (в таком случае `name` будет `NULL`):

| client\_id | name  | total\_uah |
| ---------- | ----- | ---------- |
| 1          | Anna  | 500        |
| 3          | Clara | 800        |
| 3          | Clara | 1200       |
| 5          | NULL  | 300        |


### `FULL JOIN`

```sql
SELECT clients.client_id, clients.name, orders.total_uah
FROM clients
FULL JOIN orders
   ON clients.client_id = orders.client_id;
```

Покажет всех клиентов и все заказы — даже если **нет соответствия** между таблицами:

| client\_id | name  | total\_uah |
| ---------- | ----- | ---------- |
| 1          | Anna  | 500        |
| 2          | Boris | NULL       |
| 3          | Clara | 800        |
| 3          | Clara | 1200       |
| 4          | David | NULL       |
| 5          | NULL  | 300        |

## JOIN на 3 таблицах: `clients`, `orders`, `order_items`

**`clients`**

| client\_id | name  |
| ---------- | ----- |
| 1          | Anna  |
| 2          | Boris |
| 3          | Clara |
| 4          | Dana  |

**`orders`**

| order\_id | client\_id | total\_uah |
| --------- | ---------- | ---------- |
| 101       | 1          | 500        |
| 102       | 3          | 800        |
| 103       | 3          | 1200       |
| 104       | NULL       | 250        |

**`order_items`**

| order\_id | product\_name | qty | price\_uah |
| --------- | ------------- | --- | ---------- |
| 101       | Тетрадь       | 2   | 50         |
| 101       | Ручка         | 1   | 30         |
| 102       | Пенал         | 1   | 300        |
| 999       | Ластик        | 1   | 15         |

> Здесь есть:
>
> * клиент **без заказов** (`Dana`)
> * заказ без клиента (`order_id = 104`)
> * заказ, у которого **нет товаров**
> * товар, не связанный с заказом (`order_id = 999`)

### INNER JOIN (только совпадения)

```sql
SELECT clients.name, orders.order_id, order_items.product_name
FROM clients
JOIN orders
   ON clients.client_id = orders.client_id
JOIN order_items
   ON orders.order_id = order_items.order_id;
```

**Результат:**

| name  | order\_id | product\_name |
| ----- | --------- | ------------- |
| Anna  | 101       | Тетрадь       |
| Anna  | 101       | Ручка         |
| Clara | 102       | Пенал         |

Только те клиенты, у которых **есть заказы**, и только заказы, у которых **есть товары**

---

### LEFT JOIN (все клиенты, даже без заказов)

```sql
SELECT clients.name, orders.order_id, order_items.product_name
FROM clients
LEFT JOIN orders
    ON clients.client_id = orders.client_id
LEFT JOIN order_items
    ON orders.order_id = order_items.order_id;
```

**Результат:**

| name  | order\_id | product\_name |
| ----- | --------- | ------------- |
| Anna  | 101       | Тетрадь       |
| Anna  | 101       | Ручка         |
| Boris | NULL      | NULL          |
| Clara | 102       | Пенал         |
| Clara | 103       | NULL          |
| Dana  | NULL      | NULL          |

Все клиенты: у Boris и Dana нет заказов, Clara сделала заказ, но без товаров (`103`)

---

### RIGHT JOIN (все товары, даже без клиента)

```sql
SELECT clients.name, orders.order_id, order_items.product_name
FROM clients
RIGHT JOIN orders
    ON clients.client_id = orders.client_id
RIGHT JOIN order_items
    ON orders.order_id = order_items.order_id;
```

**Результат:**

| name  | order\_id | product\_name |
| ----- | --------- | ------------- |
| Anna  | 101       | Тетрадь       |
| Anna  | 101       | Ручка         |
| Clara | 102       | Пенал         |
| NULL  | 999       | Ластик        |

Включён даже товар, который не относится к ни одному клиенту (`Ластик`, заказ `999`)

---

### FULL JOIN (всё отовсюду)

```sql
SELECT clients.name, orders.order_id, order_items.product_name
FROM clients
FULL JOIN orders
    ON clients.client_id = orders.client_id
FULL JOIN order_items
    ON orders.order_id = order_items.order_id;
```

**Результат:**

| name  | order\_id | product\_name |
| ----- | --------- | ------------- |
| Anna  | 101       | Тетрадь       |
| Anna  | 101       | Ручка         |
| Clara | 102       | Пенал         |
| Clara | 103       | NULL          |
| NULL  | 104       | NULL          |
| NULL  | 999       | Ластик        |
| Boris | NULL      | NULL          |
| Dana  | NULL      | NULL          |

Включены:

* товары без заказов
* клиенты без заказов
* заказы без клиентов
* заказы без товаров

## Комбинированный JOIN

Необходимо:

* взять **всех клиентов** (даже без заказов) → `LEFT JOIN`
* при этом к заказам подсоединить **только те товары, которые реально есть** → `INNER JOIN`

```sql
SELECT clients.name, orders.order_id, order_items.product_name
FROM clients
LEFT JOIN orders
    ON clients.client_id = orders.client_id
INNER JOIN order_items
    ON orders.order_id = order_items.order_id;
```

Что произойдёт:

* `clients LEFT JOIN orders` — ты получишь всех клиентов и их заказы (если есть)
* затем `INNER JOIN order_items` — исключатся все строки, где нет товаров

---

⚠️ **Порядок JOIN-ов имеет значение!**

Необходимо:

* взять **все заказы** (связанные и не связанные с клиентами) → `FULL JOIN`
* но из заказов получить только товары, если они есть → `LEFT JOIN`

```sql
SELECT orders.order_id, clients.name, order_items.product_name
FROM clients
FULL JOIN orders
    ON clients.client_id = orders.client_id
LEFT JOIN order_items
    ON orders.order_id = order_items.order_id;
```

# Агрегации, фильтрация агрегатов и вложенные запросы

## `GROUP BY`: группировка данных

`GROUP BY` используется, чтобы **сгруппировать строки по значению поля**, и применить агрегатные функции к каждой группе:

| Функция           | Что делает         |
| ----------------- | ------------------ |
| `COUNT()`         | Считает количество |
| `SUM()`           | Суммирует значения |
| `AVG()`           | Среднее значение   |
| `MIN()` / `MAX()` | Минимум и максимум |

**Сколько заказов сделал каждый клиент:**

```sql
SELECT client_id, COUNT(order_id) AS order_count
FROM orders
GROUP BY client_id;
```

**Общая сумма заказов по каждому клиенту:**

```sql
SELECT orders.client_id,
       SUM(order_items.quantity * order_items.price_uah) AS total_sum
FROM orders
JOIN order_items ON orders.order_id = order_items.order_id
GROUP BY orders.client_id;
```

## `HAVING`: фильтрация по агрегатам

* `WHERE` используется до агрегации — для фильтрации строк
* `HAVING` — после `GROUP BY`, чтобы фильтровать группы

**Клиенты, у которых больше 3 заказов:**

```sql
SELECT client_id, COUNT(order_id) AS order_count
FROM orders
GROUP BY client_id
HAVING COUNT(order_id) > 3;
```

**Клиенты, которые потратили более 5000 гривен:**

```sql
SELECT orders.client_id,
       SUM(order_items.quantity * order_items.price_uah) AS total_spent
FROM orders
JOIN order_items ON orders.order_id = order_items.order_id
GROUP BY orders.client_id
HAVING SUM(order_items.quantity * order_items.price_uah) > 5000;
```

## Вложенные запросы (Subqueries)

Вложенный запрос — это `SELECT`, который находится внутри другого запроса.

Типы:

* В `WHERE` (фильтрация по подмножеству)
* В `FROM` (как временная таблица)
* В `SELECT` (расчёт значения)

**Найти клиентов, которые сделали больше заказов, чем среднее по всем:**

```sql
SELECT client_id, COUNT(order_id) AS order_count
FROM orders
GROUP BY client_id
HAVING COUNT(order_id) > (
    SELECT AVG(order_count) FROM (
        SELECT client_id, COUNT(order_id) AS order_count
        FROM orders
        GROUP BY client_id
    ) AS subquery
);
```

**Найти заказы, где сумма превышает 1000 UAH:**

```sql
SELECT orders.order_id,
       SUM(order_items.quantity * order_items.price_uah) AS order_total
FROM orders
JOIN order_items ON orders.order_id = order_items.order_id
GROUP BY orders.order_id
HAVING SUM(order_items.quantity * order_items.price_uah) > 1000;
```

## Зачем это нужно в аналитике?

| Вопрос продукта                              | Как решаем                                  |
| -------------------------------------------- | ------------------------------------------- |
| Какие клиенты приносят больше всего выручки? | `GROUP BY client_id + SUM(...) + HAVING`    |
| Сколько заказов в среднем делает клиент?     | `GROUP BY client_id + COUNT(order_id)`      |
| Есть ли клиенты с большим средним чеком?     | `AVG(...)`, подзапрос на сумму / количество |
| Какие товары продаются чаще других?          | `GROUP BY product + SUM(quantity)`          |
